In [ ]:
!wget https://ai2-public-datasets.s3.amazonaws.com/sciq/SciQ.zip

--2021-10-13 19:38:05--  https://ai2-public-datasets.s3.amazonaws.com/sciq/SciQ.zip
Resolving ai2-public-datasets.s3.amazonaws.com (ai2-public-datasets.s3.amazonaws.com)... 52.218.196.114
Connecting to ai2-public-datasets.s3.amazonaws.com (ai2-public-datasets.s3.amazonaws.com)|52.218.196.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2821345 (2.7M) [binary/octet-stream]
Saving to: ‘SciQ.zip’

SciQ.zip            100%[===================>]   2.69M  4.52MB/s    in 0.6s    

2021-10-13 19:38:06 (4.52 MB/s) - ‘SciQ.zip’ saved [2821345/2821345]



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/SciQ.zip 

Archive:  /content/SciQ.zip
   creating: SciQ dataset-2 3/
  inflating: SciQ dataset-2 3/license.txt  
   creating: __MACOSX/
   creating: __MACOSX/SciQ dataset-2 3/
  inflating: __MACOSX/SciQ dataset-2 3/._license.txt  
  inflating: SciQ dataset-2 3/readme.txt  
  inflating: __MACOSX/SciQ dataset-2 3/._readme.txt  
  inflating: SciQ dataset-2 3/test.json  
  inflating: __MACOSX/SciQ dataset-2 3/._test.json  
  inflating: SciQ dataset-2 3/train.json  
  inflating: __MACOSX/SciQ dataset-2 3/._train.json  
  inflating: SciQ dataset-2 3/valid.json  
  inflating: __MACOSX/SciQ dataset-2 3/._valid.json  
  inflating: __MACOSX/._SciQ dataset-2 3  


In [ ]:
import os
os.mkdir('/content/train_para/')
os.mkdir('/content/train_qa/')

1.   Replace numbers and dashes/special characters with ''
2.   Remove lines with Figure




In [ ]:
WORKING_DIR = '/content/'

import json 

def remove_space(x):
	return x.lstrip(' ')

def get_sentences(p):
  nlp = spacy.load('en')
  tokens = nlp(p)
  return [sent.string.strip() for sent in tokens]

import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"
digits = "([0-9])"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

def add_newline(x):
  return x + '\n'

def clean_text(p):
  numbers = "[0-9]"
  special = "\W+"
  p = re.sub(numbers, " ", p)
  p = re.sub(special, " ", p)

  return p


with open('/content/SciQ dataset-2 3/train.json') as f :
    data = json.load(f)
    #print(data[0])
    count = 0
    full_set = []
    for para in data :
        # support = para['support'].replace('.','\n.').split('.')
        print(para['support'])
        support = split_into_sentences(para['support'])

        for i in support:
          if 'figure' in i.lower():
            continue
        
        support = [clean_text(i) for i in support]
        support = list(map(add_newline, support))
        with open(f'/content/train_para/{count}.txt', 'w') as s :
            s.writelines(support)
        count += 1
        full_set = full_set + support
        break
    print(full_set[0:5])

kb = ''.join(full_set)
# print(len(kb))
with open('/content/train_qa/kb.txt', 'w') as f:
  f.write(kb)


Mesophiles grow best in moderate temperature, typically between 25°C and 40°C (77°F and 104°F). Mesophiles are often found living in or on the bodies of humans or other animals. The optimal growth temperature of many pathogenic mesophiles is 37°C (98°F), the normal human body temperature. Mesophilic organisms have important uses in food preparation, including cheese, yogurt, beer and wine.
['Mesophiles grow best in moderate temperature typically between C and C F and F \n', 'Mesophiles are often found living in or on the bodies of humans or other animals \n', 'The optimal growth temperature of many pathogenic mesophiles is C F the normal human body temperature \n', 'Mesophilic organisms have important uses in food preparation including cheese yogurt beer and wine \n']


In [ ]:
text = 'Figure below shows an example of each type of fatty acid'

In [ ]:
print(len(full_set),count)

4 1


In [ ]:
df = 

with open('/content/SciQ dataset-2 3/train.json') as f :
    data = json.load(f)
    #print(data[0])
    count = 0
    qanda = []
    for para in data :
        print(para)
        count += 1
        if count == 13:
          break
        
    


{'question': 'What type of organism is commonly used in preparation of foods such as cheese and yogurt?', 'distractor3': 'viruses', 'distractor1': 'protozoa', 'distractor2': 'gymnosperms', 'correct_answer': 'mesophilic organisms', 'support': 'Mesophiles grow best in moderate temperature, typically between 25°C and 40°C (77°F and 104°F). Mesophiles are often found living in or on the bodies of humans or other animals. The optimal growth temperature of many pathogenic mesophiles is 37°C (98°F), the normal human body temperature. Mesophilic organisms have important uses in food preparation, including cheese, yogurt, beer and wine.'}
{'question': 'What phenomenon makes global winds blow northeast to southwest or the reverse in the northern hemisphere and northwest to southeast or the reverse in the southern hemisphere?', 'distractor3': 'tropical effect', 'distractor1': 'muon effect', 'distractor2': 'centrifugal effect', 'correct_answer': 'coriolis effect', 'support': 'Without Coriolis Effe

In [ ]:
with open('/content/SciQ dataset-2 3/train.json') as f :
    data = json.load(f)
    #print(data[0])
    count = 0
    qanda = []
    for para in data :
        qa = para['question'] + '\t' + para['correct_answer'] + '\n'
        
        qanda.append(qa)
    print(qanda[0:5])

with open('/content/train_qa/QA.txt', 'w') as m :
    m.writelines(qanda)

with open('/content/train_qa/QA.tsv', 'w') as m :
    m.writelines(qanda)


['What type of organism is commonly used in preparation of foods such as cheese and yogurt?\tmesophilic organisms\n', 'What phenomenon makes global winds blow northeast to southwest or the reverse in the northern hemisphere and northwest to southeast or the reverse in the southern hemisphere?\tcoriolis effect\n', 'Changes from a less-ordered state to a more-ordered state (such as a liquid to a solid) are always what?\texothermic\n', 'What is the least dangerous radioactive decay?\talpha decay\n', 'Kilauea in hawaii is the world’s most continuously active volcano. very active volcanoes characteristically eject red-hot rocks and lava rather than this?\tsmoke and ash\n']


In [ ]:
!pip install spacy

In [ ]:
##Testing ner
import spacy
import random

nlp = spacy.load('en_core_web_sm')
  
sentence = "What type of organism is commonly used in preparation of foods such as cheese and yogurt"
  
doc = nlp(sentence)

for token in doc:
  print(token, token.pos_)
    
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [ ]:
indices = list(range(1, 11677))
test_indices = random.sample(indices, 3000)
val_indices = random.sample(list(set(indices) - set(test_indices)), 2000)
print(len(indices),len(test_indices),len(val_indices))

11676 3000 2000


In [ ]:
import random
from tqdm import tqdm

def clean_text2(p):
  numbers = "[0-9]"
  special = "\W+"
  p = re.sub(numbers, "", p)
  p = re.sub(special, "", p)

  return p

def Intersection(lst1, lst2):
    return list(set(lst1).intersection(lst2))


with open('/content/drive/MyDrive/Libkge/data/aristo/entities.txt') as f:
  entity_list = f.readlines()
  entity_list = list(map(lambda x: x.strip("\n"), entity_list))
# print(entity_list[0:10])
# print("type" in entity_list)

max_len = 0
indices = list(range(1, 11677))
# test_indices = random.sample(indices, 3000)
val_indices = random.sample(indices, 2000)

with open('/content/SciQ dataset-2 3/train.json') as f :
    data = json.load(f)
    #print(data[0])
    count = 0
    train_ans_in_entity = 0
    count_train = 0
    count_val = 0
    count_test = 0
    qanda_train = []
    qanda_val = []
    qanda_test = []
    for para in tqdm(data) :

        if para['correct_answer'] not in entity_list:
              continue

        sentence = para['question']
        doc = nlp(sentence)

        words = [token.lemma_ for token in doc]
        # print('sentence')
        # print(sentence)
        # words = sentence.split(" ")
        words = list(map(clean_text2, words))
        max_len = max(max_len, len(words))
        # print('words')
        # print(words)
        topic_entities = Intersection(words, entity_list)
        # print('topic_entities')
        # print(topic_entities)

        # print()
        # print()
        try:
          topic_entity = topic_entities[0]

          qa = para['question'] + f' [{topic_entity}]' + '\t' + para['correct_answer'] + '\n'


          if count in val_indices:
            count_val += 1        
            qanda_val.append(qa)

          # elif count in test_indices:
          #   count_test += 1
          #   qanda_test.append(qa)
          
          else:
            count_train += 1
            qanda_train.append(qa)

        except Exception as E:
          # print(f"***{count}***")
          # print(E)
          # print(sentence)
          # print(topic_entities)
          pass

        count += 1

    print()
    print(qanda_test[0:1])
    print(count_train, count_val, count_test)

# with open('/content/train_qa/QA.txt', 'w') as m :
#     m.writelines(qanda)

# with open('/content/train_qa/QA.tsv', 'w') as m :
#     m.writelines(qanda)
print(train_ans_in_entity) 
print(max_len)


In [ ]:
print(qanda_val[:5])

In [ ]:
with open('/content/train_qa/QA_reduced.txt', 'w') as m :
    m.writelines(qanda_train)

with open('/content/train_qa/QA_val.txt', 'w') as m :
    m.writelines(qanda_val)


In [ ]:
import os
os.mkdir("/content/test_qa")

In [ ]:
with open('/content/SciQ dataset-2 3/test.json') as f :
    data = json.load(f)
    #print(data[0])
    count = 0
    qanda_test = []
    ans_choices_test = []
    questions = []
    for para in tqdm(data) :
        # print(para)
        if para['correct_answer'] not in entity_list:
              continue

        sentence = para['question']
        doc = nlp(sentence)

        words = [token.lemma_ for token in doc]
        # print('sentence')
        # print(sentence)
        # words = sentence.split(" ")
        words = list(map(clean_text2, words))
        max_len = max(max_len, len(words))
        # print('words')
        # print(words)
        topic_entities = Intersection(words, entity_list)
        # print('topic_entities')
        # print(topic_entities)

        # print()
        # print()
        try:
          topic_entity = topic_entities[0]

          qa = para['question'] + f' [{topic_entity}]' + '\t' + para['correct_answer'] + '\n'
          questions.append(para['question'] + f' [{topic_entity}]' + "\n")
          # print(qa)
          choices = para['distractor1'] + '\t' + para['distractor2'] + '\t' + para['distractor3'] + '\t' + para['correct_answer'] + '\n'
          # if count in val_indices:
          #   count_val += 1        
          #   qanda_val.append(qa)

          # elif count in test_indices:
          #   count_test += 1
          #   qanda_test.append(qa)
          
          # else:
          #   count_train += 1
          ans_choices_test.append(choices)
          qanda_test.append(qa)

        except Exception as E:
          # print(f"***{count}***")
          # print(E)
          # print(sentence)
          # print(topic_entities)
          pass

        count += 1
    # print(qanda_test[0:1])
    # print(len(qanda_test))
    print(para)
with open('/content/test_qa/QA_test.txt', 'w') as m :
    m.writelines(qanda_test)

with open('/content/test_qa/questions.txt', 'w') as m :
    m.writelines(questions)

with open('/content/test_qa/QA_test.tsv', 'w') as m :
    m.writelines(qanda_test)

with open('/content/test_qa/QA_choices.txt', 'w') as m :
    m.writelines(ans_choices_test)
    print(ans_choices_test[0])

100%|██████████| 1000/1000 [00:02<00:00, 379.94it/s]

{'question': 'In what form is atmospheric sulfur found?', 'distractor3': 'sulfur monoxide', 'distractor1': 'formaldehyde', 'distractor2': 'sulfuric acid', 'correct_answer': 'sulfur dioxide (so2)', 'support': 'On land, sulfur is deposited in four major ways: precipitation, direct fallout from the atmosphere, rock weathering, and geothermal vents (Figure 20.17). Atmospheric sulfur is found in the form of sulfur dioxide (SO2), and as rain falls through the atmosphere, sulfur is dissolved in the form of weak sulfuric acid (H2SO4). Sulfur can also fall directly from the atmosphere in a process called fallout. Also, as sulfur-containing rocks weather, sulfur is released into the soil. These rocks originate from ocean sediments that are moved to land by the geologic uplifting of ocean sediments. Terrestrial ecosystems can then make use of these soil sulfates (SO42-), which enter the food web by being taken up by plant roots. When these plants decompose and die, sulfur is released back into th

In [ ]:
!cp /content/test_qa/QA_test.txt /content/drive/MyDrive/EmbedKGQA/data/QA_data/QA_test.txt
!cp /content/test_qa/QA_choices.txt /content/drive/MyDrive/EmbedKGQA/data/QA_data/QA_choices.txt

In [ ]:
distractors = []
ans_not_in_entity = 0
correct_not_in_entity = 0
correct_in_entity = 0
with open('/content/drive/MyDrive/EmbedKGQA/data/QA_choices.txt') as f:  
  choices = f.readlines()
  # print(choices)
  for choice in choices:
    choice = choice.strip("\n").split('\t')
    ans_string = f"{choice[0]} {choice[1]} {choice[2]} {choice[3]}"
    sentence = ans_string
    doc = nlp(sentence)
    words = [token.lemma_ for token in doc]
    # print(words)
    # print(entity_list[:10])
    if words[3] not in entity_list:
      continue

    # distractors.append([choice[0], choice[1], choice[2], choice[3]])
print(ans_not_in_entity)
print(correct_in_entity)
print(correct_not_in_entity)
# print(distractors)

In [ ]:
[], []

In [ ]:
def get_questions_with_ent(questionA, entity_list):
  removed_tags = ['ADJ', 'DET', 'PRON', 'CONJ', 'CCONJ', 'AUX', 'ADP', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'PART', 'ADV']
  # print(questionA)
  question = questionA.split('[')[0]
  ans = questionA.split('\t')[-1]
  doc = nlp(question)
  # print(sentence)
  words = [token.lemma_ for token in doc]
  # print('sentence')
  # print(sentence)
  words = clean_text2(list2sent(words))
  # print(words)
  topic_entities = Intersection_substr(words, entity_list)
  # print(topic_entities)
  topic_sent = list2sent(topic_entities)

  # doc = nlp(topic_sent)
  # # topic_entities = [(str(entity), entity.pos_ ) for entity in doc]
  # # print(topic_entities)
  # topic_entities = [str(entity) for entity in doc if entity.pos_ not in removed_tags]
  # topic_entities = list(set(topic_entities))

  #multi word entities
  tes = []
  for te in topic_entities:
    if " " in te:
      doc = nlp(te)
      temp = [(str(entity), entity.pos_) for entity in doc if entity.pos_ not in removed_tags]
      # print(temp)
      if temp:
        tes.append(te)
        # Flag = True
  # print(list(set(topic_entities) - set(tes)))
  topic_sent = list2sent(list(set(topic_entities) - set(tes)))

  doc = nlp(topic_sent)
  te = [str(entity) for entity in doc if entity.pos_ not in removed_tags]
  te = list(set(te))
  
  tes = tes + te

  topic_entities = Intersection(topic_entities, tes)

  ques_template = f"{question} [te]"
  questions = []
  for te in topic_entities:
    # if te == 'metropolitan area':
    #   print(topic_entities)
    #   print(ques_template)
    questions.append(ques_template.replace('[te]', f'[{te}]'))

  return questions, ans

In [ ]:
def get_ans(questions, ans):

  for ques in questions:
    temp_ans = get_prediction(model, ques, e, entity2idx, idx2entity, embedding_matrix)
    if temp_ans == ans:
      return True
  return False

In [ ]:
import os
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.nn import functional as F
from dataloader import DatasetMetaQA, DataLoaderMetaQA
from model import RelationExtractor
import networkx as nx
from kge.model import KgeModel
from kge.util.io import load_checkpoint

def getEntityEmbeddings(kge_model):
    e = {}
    entity_dict = f'../../pretrained_models/embeddings/commonsense/entity_ids.del'
    embedder = kge_model._entity_embedder
    f = open(entity_dict, 'r')
    for line in f:
        line = line[:-1].split('\t')
        ent_id = int(line[0])
        ent_name = line[1]
        e[ent_name] = embedder._embeddings(torch.LongTensor([ent_id]))[0]
    f.close()
    return e

def prepare_embeddings(embedding_dict):
    entity2idx = {}
    idx2entity = {}
    i = 0
    embedding_matrix = []
    for key, entity in embedding_dict.items():
        entity2idx[key] = i
        idx2entity[i] = key
        i += 1
        embedding_matrix.append(entity)
    return entity2idx, idx2entity, embedding_matrix


embedding_dim = 256
relation_dim = 50
model_name = 'ComplEx'
do_batch_norm = 1

def load_models(load_from):
    
    #Load KG Embeddings
    checkpoint_file = f'../../pretrained_models/embeddings/{load_from}/checkpoint_best.pt'
    print('Loading kg embeddings from', checkpoint_file)
    kge_checkpoint = load_checkpoint(checkpoint_file)
    kge_model = KgeModel.create_from(kge_checkpoint)
    kge_model.eval()
    e = getEntityEmbeddings(kge_model)

    print('Loaded entities and relations')

    entity2idx, idx2entity, embedding_matrix = prepare_embeddings(e)


    #Load RE Model
    model = RelationExtractor(embedding_dim=embedding_dim, num_entities = len(idx2entity), relation_dim=relation_dim, 
                              pretrained_embeddings=embedding_matrix, device=device, 
                              model = model_name, do_batch_norm=do_batch_norm)
      
    fname = "/content/drive/MyDrive/EmbedKGQA_final/output/" + load_from + ".pt"
    print('Loading from %s' % fname)
    model.load_state_dict(torch.load(fname, map_location=torch.device('cpu')))
    print('Loaded successfully!')
    model.to(device)

    return model, e, entity2idx, idx2entity, embedding_matrix

def get_prediction(model, question, e, entity2idx, idx2entity, embedding_matrix):

  #Getting head and question
  # print(question)
  question_t = question.split('\t')[0]
  # print(question)
  question = question.split('[')
  question_1 = question[0]
  question_2 = question[1].split(']')
  head = question_2[0].strip()
  question_2 = question_2[1]
  question = question_1+'NE'+question_2
  # print(question, head)
  head = entity2idx[head.strip()]
  head = torch.tensor(head, dtype=torch.long)
  head = head.to(device)
  data = [head, question]
  # Get_prediction
  dataloader = DatasetMetaQA(data, e, entity2idx)
  
  question_tokenized, attention_mask = dataloader.tokenize_question(question)
  question_tokenized = question_tokenized.to(device)
  attention_mask = attention_mask.to(device)

  # print(head)
  scores = model.get_score_ranked(head=head, question_tokenized=question_tokenized, attention_mask=attention_mask)[0]
  mask = torch.zeros(len(entity2idx)).to(device)
  mask[head] = 1
  #reduce scores of all non-candidates
  new_scores = scores - (mask*99999)
  pred_ans = torch.argmax(new_scores).item()

  return idx2entity[pred_ans]

In [ ]:
# model, e, entity2idx, idx2entity, embedding_matrix = load_models('aristo_final')
from tqdm import tqdm

quesAns = get_questions()[0]
correct = 0
total = 0
for questionA in tqdm(quesAns):
  questions, ans = get_questions_with_ent(questionA, entity_list)
  # print(questions)
  if get_ans(questions, ans):
    correct += 1
  total += 1
  if total % 50 == 0:
    print(correct)
  # print(questions)
  # print(ans)
  # print("**next**")
print(f"{correct/total}%")